<H3>PRI 2023/24: second
    project delivery</H3>

**GROUP 11**
- Francisco Martins, 99068
- Tunahan Güneş, 108108
- Sebastian Weidinger, 111612

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
from G11_code.data_collection import *
from G11_code.helper_functions import *
from G11_code.indexing import *
from G11_code.evaluation import *
from G11_code.clustering import *
from G11_code.supervised_classification import *

In [ ]:
article_path = os.path.join("..", "BBC News Summary", "BBC News Summary", "News Articles")
summary_path = os.path.join("..", "BBC News Summary", "BBC News Summary", "Summaries")
print("Article path:", article_path)
print("Summary path:", summary_path)
_article_file_paths_by_cat, _articles_by_cat, _summary_file_paths_by_cat, _summaries_by_cat, category_names = read_files(article_path, summary_path)

In [ ]:
_summary_sentence_indices_by_cat, faulty_summary_ids = get_summary_sentence_indices(_articles_by_cat, _summaries_by_cat)

In [ ]:
articles_by_cat = remove_entries(_articles_by_cat, faulty_summary_ids)
articles = flatten(articles_by_cat)
article_file_paths_by_cat = remove_entries(_article_file_paths_by_cat, faulty_summary_ids)
article_file_paths = flatten(article_file_paths_by_cat)
summaries_by_cat = remove_entries(_summaries_by_cat, faulty_summary_ids)
summaries = flatten(summaries_by_cat)
summary_file_paths_by_cat = remove_entries(_summary_file_paths_by_cat, faulty_summary_ids)
summary_file_paths = flatten(summary_file_paths_by_cat)
summary_sentence_indices_by_cat = remove_entries(_summary_sentence_indices_by_cat, faulty_summary_ids)
summary_sentence_indices = flatten(summary_sentence_indices_by_cat)

In [ ]:
dict_path_to_articleID = {path:i for i, path in enumerate(article_file_paths)}
def map_path_to_articleID(path):
    path = os.path.normpath(path)
    return dict_path_to_articleID.get(path)

In [ ]:
I = indexing(articles)

In [ ]:
path = os.path.join(article_path, 'tech', '199.txt')
d = map_path_to_articleID(path)

In [ ]:
print(I.doc_to_string(d))
dissimilarity_matrix_tfidf = tf_idf_compute_dissimilarity_matrix(d, I)
plt.matshow(dissimilarity_matrix_tfidf)

In [ ]:
# If using another function to map the similarity matrix into a dissimilarity matrix such as d_ij = (1-s_ij^0.5)^2
plt.matshow((1-np.power(1 - dissimilarity_matrix_tfidf, 1/2))**2)

In [ ]:
ratios = np.array([len(nltk.sent_tokenize(articles[i]))/len(summary_sentence_indices[i]) for i in range(len(articles))])
plt.boxplot(ratios)
None

In [ ]:
pretrained_weights = 'distilbert-base-uncased'
bert_tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)
bert_model = DistilBertModel.from_pretrained(pretrained_weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

In [ ]:
sentence_embeddings_path = './embeddings/sentence_embeddings.pkl'
bert_params = (bert_tokenizer, bert_model, device)
dissimilarity_matrix_bert = bert_compute_dissimilarity_matrix(0, articles, bert_params, sentence_embeddings_path)

In [ ]:
plt.matshow(dissimilarity_matrix_bert)

In [ ]:
# print two closest sentences according to bert
for d in (0, 10, 600, 610, 900, 910):
    bert_params = (bert_tokenizer, bert_model, device)
    dissimilarity_matrix_bert = bert_compute_dissimilarity_matrix(d, articles, bert_params)
    index_closest_bert = np.argmin(dissimilarity_matrix_bert)
    tokenized_article = nltk.sent_tokenize(articles[d])
    num_sent = len(tokenized_article)
    sent1 = index_closest_bert//num_sent
    sent2 = index_closest_bert%num_sent
    print(tokenized_article[sent1])
    print(tokenized_article[sent2])
    print(dissimilarity_matrix_bert[sent1, sent2])
    print('---')

In [ ]:
# print two closest sentences according to tfidf
for d in (0, 10, 600, 610, 900, 910):
    dissimilarity_matrix_tfidf = tf_idf_compute_dissimilarity_matrix(d, I)
    index_closest_tfidf = np.argmin(dissimilarity_matrix_tfidf)
    tokenized_article = nltk.sent_tokenize(articles[d])
    num_sent = len(tokenized_article)
    sent1 = index_closest_tfidf//num_sent
    sent2 = index_closest_tfidf%num_sent
    print(tokenized_article[sent1])
    print(tokenized_article[sent2])
    print(dissimilarity_matrix_tfidf[sent1, sent2])
    print('---')

In [ ]:
# save sentence and document embeddings 
pretrained_weights = 'distilbert-base-uncased'
bert_tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)
bert_model = DistilBertModel.from_pretrained(pretrained_weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)
path = './embeddings'
if not os.listdir(path): 
    save_embeddings(articles, bert_tokenizer, bert_model, device, path)
else: 
    print(f'Files exist in folder {path}')

In [ ]:
# load embeddings 
sentence_embeddings_path = os.path.join('./embeddings', 'sentence_embeddings.pkl')
sentence_embeddings_by_cat = pickle_load(sentence_embeddings_path)
document_embeddings_path = os.path.join('./embeddings', 'document_embeddings.pkl')
document_embeddings_by_cat = pickle_load(document_embeddings_path)

In [ ]:
from G11_code.training import *

train_emb_by_cat, test_emb_by_cat, train_ind_by_cat, test_ind_by_cat = split_by_cat(sentence_embeddings_by_cat, summary_sentence_indices_by_cat)

X_train, Y_train = get_XY(train_emb_by_cat, train_ind_by_cat)
X_test, Y_test = get_XY(test_emb_by_cat, test_ind_by_cat)
X_train = flatten(X_train)
X_test = flatten(X_test)
Y_train = flatten(Y_train)
Y_test = flatten(Y_test)

In [ ]:
n_components = 60

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
X_train_t = reducer.fit(np.array(X_train))

In [ ]:
pca = fit_PCA(X_train, n_components=n_components)
X_train_trans = transform_PCA(pca, X_train)
X_test_trans = transform_PCA(pca, X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

#model = training(train_emb_by_cat, train_ind_by_cat, model_name="XGBoost", use_pca=True, n_components=n_components)
model = training(train_emb_by_cat, train_ind_by_cat, model_name="XGBoost")

predictions = model.predict(X_test)

accuracy = accuracy_score(Y_test, predictions)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.75      0.68      4970
           1       0.52      0.37      0.43      3609

    accuracy                           0.59      8579
   macro avg       0.57      0.56      0.55      8579
weighted avg       0.58      0.59      0.57      8579

In [ ]:
#precision, recall, auc = supervised_evaluation(test_emb_by_cat, test_ind_by_cat, model, model_name="XGBoost", use_pca=True, X_train=X_train, n_components=n_components)
precision, recall, auc = supervised_evaluation(test_emb_by_cat, test_ind_by_cat, model, model_name="XGBoost")
print(f'Classifier XGBoost - Precision: {precision} | Recall: {recall} | AUC: {auc}')

In [ ]:
# problem with variable output
# how can be every sentence in input sequence be classified? 

from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Bidirectional

LSTM_units = 100

model = Sequential()
model.add(LSTM(units=LSTM_units, 
               input_shape=(None, 768), 
               return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])
#print(model.summary())


In [ ]:
X_train, Y_train = get_XY(train_emb_by_cat, train_ind_by_cat)
pbar = tqdm(total=len(X_train))
for X, y in tqdm(zip(X_train, Y_train), total=len(X_train)):
    #X = np.array(transform_PCA(pca, X))
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    y = np.array(y)
    y = y.reshape(1, len(y), 1)
    model.fit(X, y, epochs=1, batch_size=1, verbose=0)
    pbar.update(1)

In [ ]:
X_test, y_test = get_XY(test_emb_by_cat, test_ind_by_cat)

for X, y in zip(X_test, y_test): 
    #X = np.array(transform_PCA(pca, X))
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    #y = np.array(y)
    #y = y.reshape(1, len(y), 1)
    y_hat = model.predict(X)
    print(y_hat)
    t = 0

In [ ]:
model = training(train_emb_by_cat, train_ind_by_cat, model_name="NN", use_pca=True, n_components=n_components)
predictions = np.round(model.predict(np.array(X_test_trans)))
print(classification_report(Y_test, predictions))

In [ ]:
loss_and_metric = model.evaluate(np.array(X_test_trans), np.array(Y_test))
print(loss_and_metric)

In [ ]:
predictions = model.predict(np.array(X_test_trans))
print(np.round(predictions))

In [ ]:
precision, recall, auc = supervised_evaluation(test_emb_by_cat, test_ind_by_cat, model, model_name="NN", use_pca=True, X_train=X_train, n_components=n_components)
print(f'Classifier NN - Precision: {precision} | Recall: {recall} | AUC: {auc}')

<H2>Main facilities</H2>

<h3>Part I: clustering</h3>

*A) Clustering*

In [ ]:
#code, statistics and/or charts here

*B) Summarization*

In [ ]:
#code, statistics and/or charts here

*C) Keyword extraction*

In [ ]:
#code, statistics and/or charts here

*D) Evaluation*

In [ ]:
#code, statistics and/or charts here

<h3>Part II: classification</h3>

*A) Feature extraction*

In [ ]:
# load embeddings 
sentence_embeddings_path = os.path.join('./embeddings', 'sentence_embeddings.pkl')
sentence_embeddings_by_cat = pickle_load(sentence_embeddings_path)
sentence_embeddings = flatten(sentence_embeddings_by_cat)
document_embeddings_path = os.path.join('./embeddings', 'document_embeddings.pkl')
document_embeddings_by_cat = pickle_load(document_embeddings_path)
document_embeddings = flatten(document_embeddings_by_cat)

In [ ]:
import pandas as pd
data = get_dataframe(docs=range(len(articles)), sent_embeddings=sentence_embeddings, doc_embeddings=document_embeddings,  I=I, article_file_paths=article_file_paths, articles=articles, k=0.2, b=0.75, p_keywords=10)
df = pd.DataFrame(data)
print(df.shape)
df

*B) Classification*

In [ ]:
#code here

*C) Ranking extension*

In [ ]:
#code here

*D) Evaluation*

In [ ]:
#code, statistics and/or charts here

<H2>Question materials (optional)</H2>

<H3>Part I: clustering</H3>

**(1)** Do clustering-guided summarization alters the behavior and efficacy of the IR system?

In [ ]:
#code, statistics and/or charts here

**(2)** How sentence representations, clustering choices, and rank criteria impact summarization?

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>END</H3>